In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array,
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Flatten, Dense
from keras.utils.training_utils import multi_gpu_model
import csv
import cv2
import numpy as np
import random as rand

In [ ]:
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

In [ ]:
# We define in advance constants to build the model

INPUT_SHAPE = (96, 96, 3)
OUTPUT_SIZE = 2

LEARNING_RATE = 0.01
OPTIMIZER = keras.optimizers.Adam()
LOSS = 'binary_crossentropy'
METRIC = 'accuracy'

EPOCHS = 100
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 50

MODEL_PATH = "./binary_classifier/net_0_model.json"
WEIGHTS_PATH = "./binary_classifier/net_0_weights.h5"

In [ ]:
def generator(file_name, epoch_act, epoch_tot, batch_size):
    csv_file = open(file_name)
    reader = csv.reader(csv_file)
    batch_count = 0
    patched_images = []
    for line in reader:
        patches = []
        classes = []
        image_name = line[3]                    # line[3] is path to image
        if image_name not in patched_images:    #open image only if not opened yet
            patched_images.append(image_name)
            image = cv2.imread(image_name) 
        if line[2] == "background":
            patches.append(extract_background_patch(image, line[0], line[1]))
            classes.append("background")    
        elif line[2] == "sea lion":
            patches.append(extract_sea_lion_patch(image, line[0], line[1], epoch_act, epoch_tot))
            classes.append("sea lion")
        batch_count += 1
        if batch_count >= batch_size:
            batch_count = 0
            x = np.array(patches)
            y = np.array(classes)
            yield x, y

In [ ]:
# Build parallel model (multi gpu)

model = Sequential()
# First layer
model.add(Convolution2D(8, (5, 5), activation='relu', padding='valid', input_shape=INPUT_SHAPE))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# Second layer
model.add(Convolution2D(5, (3, 3), activation='relu', padding='valid'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# Third layer
model.add(Convolution2D(5, (3, 3), activation='relu', padding='valid'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# Fourth layer
model.add(Convolution2D(10, (3, 3), activation='relu', padding='valid'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

model.add(Flatten())

# model.add(Dropout(0.5))
model.add(Dense(OUTPUT_SIZE, activation='softmax'))

parallel_model = multi_gpu_model(model, gpus=2)
parallel_model.compile(loss=LOSS, optimizer=OPTIMIZER, metrics=[METRIC])

In [ ]:
#generation of augmented datasets
#augmented training dataset
train_datagen = ImageDataGenerator(
    rotation_range=360, #range for random rotations
    #width_shift_range=,#float of range for random hor. shifts
    #height_shift_range=,#float of range for random ver. shifts
    horizontal_flip=1,  #random hor.flip
    vertical_flip=1, #random ver. flip
    #rescale=
)

#not augmented testing dataset
test_datagen = ImageDataGenerator()

#not augmented validation dataset
validation_dataset = ImageDataGenerator()

In [ ]:
#this method needs a directory with training data and another one with validation ones
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

In [ ]:
#model.fit_generator(
#        train_generator,
        #steps_per_epoch= // batch_size, #n. of samples of dataset divided by the batch size
#        epochs=50,
#        validation_data=validation_generator,
#        validation_steps=800 // batch_size)
#model.save_weights('weights.h5')  

In [ ]:
# Create data generator for augmented images

datagen = ImageDataGenerator(
    # Define rotation range between 0 and 360 degrees
    rotation_range=360,
    # Define range of random horizontal shifts (expressed as fraction of total width)
    width_shift_range=0.2,
    # Define range of random vertical shifts (expressed as fraction of total height)
    height_shift_range=0.2,
    # Define shear intensity
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest')

In [ ]:
# Background patches extractor
def extract_background_patch(image, coord_x, coord_y):
    patch = image[coord_y-48:coord_y+48, coord_x-48:coord_x+48, :]
    return patch

In [ ]:
# Sea lions patches extractor
def extract_sea_lion_patch(image, coord_x, coord_y, epoch, n_epochs):
    if coord_x < 72:
        coord_x = 0
    elif coord_x > len(image[0]) - 72:
        coord_x = len(image[0]) - 144
    else:
        coord_x = coord_x - 72
    if coord_y < 72:
        coord_y = 0
    elif coord_y > len(image) - 72:
        coord_y = len(image) - 144
    else:
        coord_y = coord_y - 72
    external_patch = image[coord_y:coord_y+144, coord_x:coord_x+144, :]
    if rand.uniform(0.0, 1.0) < (epoch/n_epochs):
        # Perform transformation
        for aug_img in datagen.flow(img_to_array(external_patch), [1, 0]):
            external_patch = array_to_img(aug_img)
            break
    patch = external_patch[72-48:72+48, 72-48:72+48, :]
    return patch